SQL로 정제하기 어려운 데이터 처리의 경우 파이썬 판다스를 활용하기로 함

1. 날짜별 세션 로그 수와 장바구니, 구매 건수

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('data/market_final.parquet')

In [3]:
#event_time 컬럼의 날짜별로 세션 로그 수 집계
session_by_date = df.groupby(df['event_time'].dt.date)['user_session'].count().to_frame()

#event_time 컬럼의 날짜별로 장바구니 건수 집계
session_by_date['cart_cnt'] = df[df['event_type'] == 'cart'].groupby(df['event_time'].dt.date)['event_type'].count()

#event_time 컬럼의 날짜별로 구매 건수 집계
session_by_date['purchase_cnt'] = df[df['event_type'] == 'purchase'].groupby(df['event_time'].dt.date)['event_type'].count()

#인덱스 정리
session_by_date.reset_index(inplace=True)

In [4]:
session_by_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   event_time    30 non-null     object 
 1   user_session  30 non-null     int64  
 2   cart_cnt      30 non-null     int64  
 3   purchase_cnt  29 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.1+ KB


2. 요일별 세션 로그 수와 장바구니, 구매 건수

In [6]:
session_by_date['event_time'] = pd.to_datetime(session_by_date['event_time'])
session_by_date.insert(1,'weekday',session_by_date['event_time'].dt.day_name())

In [7]:
session_by_date

,event_time,weekday,user_session,cart_cnt,purchase_cnt
0,2019-11-01,Friday,897163,16156,16199.0
1,2019-11-02,Saturday,960117,16495,15450.0
2,2019-11-03,Sunday,981489,17201,15992.0
3,2019-11-04,Monday,1101022,18275,18445.0
4,2019-11-05,Tuesday,1031332,16227,16616.0
5,2019-11-06,Wednesday,1022000,16565,17277.0
6,2019-11-07,Thursday,1079592,16531,16861.0
7,2019-11-08,Friday,1175732,49665,18606.0
8,2019-11-09,Saturday,1163130,44126,16328.0
9,2019-11-10,Sunday,1203118,44274,16117.0


In [10]:
#결측치 제거 및 데이터타입 정리
session_by_date = session_by_date.fillna(0)
session_by_date = session_by_date.astype({'purchase_cnt':'int64'})

In [13]:
#parquet 파일로 저장
session_by_date.to_parquet('data/session_by_date.parquet', compression='gzip')

이 후 빅쿼리로 옮겨 시각화 및 대쉬보드 제작 진행